In [1]:
import requests, matplotlib.pyplot, json, io, codecs
from bs4 import BeautifulSoup
from tkinter import *
from PIL import Image, ImageTk
from urllib.request import urlopen

In [2]:
class Movie:
    def __init__(self):
        self.title = "title"
        self.rating = "rating"
        self.runtime = "runtime"
        self.genre = "genre"
        self.date = "date"
        self.director = "director"
        self.writer = "writer"
        self.stars = "stars"
        self.summary = "summary"
        self.tomCon = "tomCon"
        self.iRating = "iRating"
        self.iRatingCount = "iRatingCount"
        self.tRatingC = "tRatingC"
        self.tRatingCCount = "tRatingCCount"
        self.tRatingA = "tRatingA"
        self.tRatingACount = "tRatingACount"
        self.posterLink = "posterLink"
        self.sceneLink = "sceneLink"
        self.pageLink = "pageLink"

In [3]:
def RottenCheck(name, movie):
    googleURL = "https://www.google.com/search?q=" + name.replace("_", "+") + "+rotten+tomatoes"
    print(googleURL)
    pageG = requests.get(googleURL)
    soupG = BeautifulSoup(pageG.content, 'html.parser')

    tom = soupG.find(class_="kCrYT")
    tom = tom.find('a')["href"]
    tom = tom.split("&")[0].split("=")[1]
    print(tom)
  
    URLR = tom
    pageR = requests.get(URLR)
    soupR = BeautifulSoup(pageR.content, 'html.parser')
    
    movie.tomCon = soupR.find(class_= "mop-ratings-wrap__text mop-ratings-wrap__text--concensus").text

    tomRating = soupR.find_all(class_= "mop-ratings-wrap__percentage")
    tomRatingCount = soupR.find_all(class_= "mop-ratings-wrap__text--small")
    movie.tRatingCCount= tomRatingCount[1].text.strip()
    movie.tRatingACount= tomRatingCount[2].text.strip().split()[2]
    movie.tRatingC = tomRating[0].text.strip()
    movie.tRatingA = tomRating[1].text.strip()
    
    return(movie)

In [4]:
def searchMovie(name):
    movie = Movie()
    
    googleURL = "https://www.google.com/search?q=" + name.replace(" ", "+") + "+imdb"
    print(googleURL)
    pageG = requests.get(googleURL)
    soupG = BeautifulSoup(pageG.content, 'html.parser')

    imdbURL = soupG.find(class_="kCrYT")
    imdbURL = imdbURL.find('a')["href"]
    imdbURL = imdbURL.split("&")[0].split("=")[1]   
    
    page = requests.get(imdbURL)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    sceneURL = imdbURL +"mediaindex"
    pageI = requests.get(sceneURL)
    soupI = BeautifulSoup(pageI.content, 'html.parser')
    
    scene = soupI.find(class_ = "media_index_thumb_list")
    scene = scene.find('a')["href"]
    sceneURL = "https://www.imdb.com/" + scene
    print(sceneURL)
    
    pageI = requests.get(sceneURL)
    soupI = BeautifulSoup(pageI.content, 'html.parser')
    
    scene = soupI.find(class_ = "pswp__zoom-wrap")
    scene = soupI.text.split("src")
    movie.sceneLink = scene[2].split(',"w"')[0].replace(':"',"").replace('"',"")
    
    movie.title = soup.find("h1").text
    stats = soup.find(class_= "subtext")
    movie.iRating = soup.find(itemprop= "ratingValue").text
    movie.iRatingCount = soup.find(itemprop= "ratingCount").text
    movie.summary = soup.find(class_= "summary_text").text.strip()

    r = stats.text
    r = r.replace("\n", "")
    r = r.split("|")
    movie.rating = r[0].strip()
    movie.runtime = r[1].strip()
    movie.genre = r[2].strip()
    movie.date = r[3].strip()

    credits = soup.find_all(class_= "credit_summary_item")
    director = credits[0].text.strip().split()
    writer = credits[1].text.strip().split(",")
    stars = credits[2].text.strip().split(",")

    movie.director = director[1] + " " + director[2]
    movie.writer = writer[0].split("\n")[1]
    movie.stars = stars[0].split("\n")[1] + "," + stars[1] + "," + stars[2].split("|")[0]

    tm = movie.title.split("\xa0")[0].replace(" ", "_")
    movie = RottenCheck(tm, movie)
    #tTRating, tTRCount, tARating, tTACount, tomConsensus = RottenCheck(tm)
    
    titleURL = movie.title.split()
    movie.pageLink = "_".join(titleURL[:-1]).lower() + ".html"
    
    movie.posterLink = soup.find("img")["src"]
    #toHTMLHome("the_lighthouse.html", iURL, title)#moviePageLink, moviePosterLink, movieTitle
    #toHTML(title,rating,runtime,genre,date,director,writer,stars,summary,tomConsensus,userRating,userRating2,
               #tTRating,tTRCount,tARating,tTACount,iURL, scene)
    renderInfo(movie)
    

In [11]:
def toJSON(movie):
    f = open("movieDB.json", "r")
    data = f.read()
    movies = json.loads(data)

    newMovie = {}

    newMovie["title"] = movie.title
    newMovie["rating"] = movie.rating
    newMovie["runtime"] = movie.runtime
    newMovie["genre"] = movie.genre
    newMovie["date"] = movie.date
    newMovie["director"] = movie.director
    newMovie["writer"] = movie.writer
    newMovie["stars"] = movie.stars
    newMovie["summary"] = movie.summary
    newMovie["tomCon"] = movie.tomCon
    newMovie["iRating"] = movie.iRating
    newMovie["iRatingCount"] = movie.iRatingCount
    newMovie["tRatingC"] = movie.tRatingC
    newMovie["tRatingCCount"] = movie.tRatingCCount
    newMovie["tRatingA"] = movie.tRatingA
    newMovie["tRatingACount"] = movie.tRatingACount
    newMovie["tRatingACount"] = movie.tRatingACount
    newMovie["sceneLink"] = movie.sceneLink
    newMovie["pageLink"] = movie.pageLink
    newMovie["videoPageLink"] = movie.pageLink.replace("_","-")
    newMovie["videoFileLink"] = movie.pageLink.replace(".html",".mp4")
    
    movies.append(newMovie)
    f.close()

    f = open("movieDB.json", "w")
    f.write(json.dumps(movies))
    f.close()

In [12]:
def renderInfo(movie):
    
    rightX = 445
    
    image_bytes = urlopen(movie.posterLink).read()
    data_stream = io.BytesIO(image_bytes)
    pil_image = Image.open(data_stream)
    tk_image = ImageTk.PhotoImage(pil_image)

    img = Label(window, image = tk_image)
    img.place(x=rightX, y=160, anchor=CENTER)
    
    statsText = StringVar()
    lbl = Label(window, justify = LEFT ,wraplength = 350, textvariable = statsText, bg = "#3a3d3b", fg="white")
    
    statsText.set(movie.title + "\n" + 
                movie.rating + "\n" +
                movie.runtime + "\n" +
                movie.genre + "\n" +
                movie.date + "\n" + "\n" +
                "Director: " + movie.director + "\n" +
                "Writer: " + movie.writer + "\n" +
                "Stars: " + movie.stars + "\n" + "\n" +

                movie.summary + "\n" + "\n" +
                movie.tomCon + "\n" + "\n" +
                "IMDb: " + movie.iRating + " | " + movie.iRatingCount + "\n" +
                "Rotten Critics: " + movie.tRatingC + " | " + movie.tRatingCCount + "\n" +
                "Rotten Audience: " + movie.tRatingA + " | " + movie.tRatingACount + "\n") 
    
    lbl.place(x=0, y=0)
    
    searchingText = "Search for another movie"
    searchText.set(searchingText)
    
    searching.place(x=rightX, y=335, anchor=CENTER)
    txt.place(x=rightX, y=360, anchor=CENTER)
    
    htmlButton = Button(window, text = "To HTML", bg="white", bd=1, 
                        command = lambda: toJSON(movie))
    
    htmlButton.place(x=150, y=360, anchor=W)  
    
    window.update()
    window.mainloop()

In [14]:
def search(event=None):

    movie = txt.get()
    searchingText = "Searching for " + movie + " ..."
    searchText.set(searchingText)
    window.update()
    
    searchMovie(movie)

window = Tk()
window.title("Movie_DB")
window.geometry('550x400')
window.configure(bg="#3a3d3b")

searchText = StringVar()
searchText.set("Search for a movie")

searching = Label(window, justify = LEFT, wraplength = 200, textvariable = searchText, bg="#3a3d3b", fg="white")    
searching.place(x=275, y=175, anchor=CENTER)

txt = Entry(window,width=30)
txt.bind('<Return>', search)
txt.place(x=275, y=200, anchor=CENTER)
txt.focus()
window.focus_force()

window.mainloop()

In [23]:
def toHTML(title,rating,runtime,genre,date,director,writer,stars,summary,tomConsensus,userRating,userRating2,
               tTRating,tTRCount,tARating,tTACount,iURL,scene, titleURL):
    print("Makeing HTML page...")
    f = open(titleURL,'w')
    videoSource = "The_Lighthouse.mp4"
    awards = "1"
    message = """
    <!DOCTYPE html>
    <html>
        <head>
            <link rel="stylesheet" href="Home.css">
            <link href="https://vjs.zencdn.net/7.7.5/video-js.css" rel="stylesheet" />
            <script src="https://vjs.zencdn.net/ie8/1.1.2/videojs-ie8.min.js"></script>
            <title>"""+title+"""</title>
        </head>
        <body>
            <div>
                <div class="sceneWrap">
                    <img class = "scene" src ='"""+scene+"""'>
                </div>
                <p class = "stats">
                    <br>
                    <span>"""+title+"""</span> <br>
                    <br>
                    """+date+""" | """+rating+""" | """+runtime+""" <br>
                    IMDb: """+userRating.text+""" | """+userRating2.text+""" <br>
                    Rotten Critics: """+tTRating+""" | """+tTRCount+""" <br>
                    Rotten Audience: """+tARating+""" | """+tTACount+""" <br>
                    <br>
                    """+summary.text.strip()+"""<br>
                    <br>
                    Consensus:<br>
                    """+tomConsensus+"""<br>
                    <br>
                    Director: """+director+"""<br>
                    Writer:  """+writer+"""<br>
                    Stars:  """+stars+"""<br>
                    <br>
                    """+genre+"""<br>
                    <br>
                    Nominated for """+awards+""" Oscar(s)<br>

                </p>
                <a href="Home.html">
                    <button class="homeButton">
                        <img src="backarrow.png" height="36" width="36" >
                    </button>
                </a>
            </div>
            <div class = "vidWrap">
                <video
                class="video-js vjs-big-play-centered"
                controls
                preload="auto"
                width="480"
                height="270"
                data-setup="{}"
                >
                <source src='"""+videoSource+"""' type="video/mp4" />
                <p class="vjs-no-js">
                  To view this video please enable JavaScript, and consider upgrading to a
                  web browser that
                  <a href="https://videojs.com/html5-video-support/" target="_blank"
                    >supports HTML5 video</a
                  >
                </p>
                </video>
                <script src="https://vjs.zencdn.net/7.7.5/video.js"></script>
                <!--<video width="480" height="270" controls>
                        <source src='videoSource' type="video/mp4">
                </video>-->
            </div>
        </body>
    </html>
    """
    
    f.write(message)
    f.close()
    
    print("HTML page made")
    